In [21]:
import scipy.io 
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle
import csv 

**Lecture des données**

In [5]:
mat = scipy.io.loadmat("data_doc.mat")
X = mat['Xts'].T.toarray()
df_y = pd.DataFrame(mat['yts'])
y = pd.get_dummies(df_y[0]).to_numpy() # OneHotEncoding
X_to_predict = pd.DataFrame(mat['Xvr'].T.toarray())

**Traitement des données**
-> normalisation


In [6]:
scaler = StandardScaler()
X = scaler.fit_transform(X)
X_to_predict = scaler.fit_transform(X_to_predict)

In [57]:
class Net:
    def __init__(self, mots, classe):
        """Initialisation des poids et des biais """
        self.weights = np.random.randn(mots, classe)
        self.biases = np.random.randn(classe)
        
       
    def forward(self, x):
        """Forward pass pour calculer la sortie du réseau"""
        # sortie z = W^t*X+B
        z = np.dot(self.weights.T, x) + self.biases
        # fct d'activation = sigmoïde
        activation = sigmoid(z)
        return z, activation


    def evaluate_gradient(self, x, y):
        """Calcul des dérivées partielles de la loss"""
        # on récupère la valeur de z et act(z) 
        z, activation = self.forward(x)
        # calculer les dérivées partielles pour la couche de sortie par rapport à W et B
        delta_b = (activation-y)*sigmoid_prime(z)
        delta_w = np.outer(x, delta_b)
        return delta_b, delta_w


    
    def update(self, X, y, lr):
        """Mise à jour des poids et des biais en batch directement
           GD -> batch size = taille(X),
           SGD -> batch size = 1
           SGD mini batch -> batch size = size des batchs voulus"""
        
        # stocker la somme des gradients des x du batch
        somme_delta_b = np.zeros(self.biases.shape)
        somme_delta_w = np.zeros(self.weights.shape)

        for x,y in zip(X,y):
            delta_b, delta_w = self.evaluate_gradient(x, y)
            somme_delta_b+=delta_b
            somme_delta_w+=delta_w
        
        self.weights -= (lr/X.shape[0])*somme_delta_w 
        self.biases -= (lr/X.shape[0])*somme_delta_b

                                                                                                                                  
    def SGD(self, X, y, lr, epochs):
        """SGD batchsize=1"""
        # shuffle les données
        X, y = shuffle(X,y)
        for epoch in range(epochs):
            # à chaque époque on parcourt tous les X, on calcule le gradient et on met à jour les poids et le biais pour chaque X
            for i in range(len(X)):
                self.update(X[i], y[i], lr)

            # calcul de l'accuracy et de la loss pour cette époque 
            accuracy, loss = self.evaluate_model(X, y)
            print(f'Epoch {epoch}')
            print(f'Accuracy = {accuracy}')
            print(f'Loss = {loss}')

    def GD(self, X, y, lr, epochs):
        """Gradient descent"""
        X, y = shuffle(X,y)
        for epoch in range(epochs):
            # à chaque époque, on met le gradient une seule fosi à jour
            self.update(X, y, lr)

            # calcul de l'accuracy et de la loss pour cette époque 
            accuracy, loss = self.evaluate_model(X, y)
            print(f'Epoch {epoch}')
            print(f'Accuracy = {accuracy}')
            print(f'Loss = {loss}')

    
    def SGD_batch(self, X, y, lr, epochs, batch_size):
        """SGD+mini-batch"""
        # shuffle les données
        X, y = shuffle(X,y)
        for epoch in range(epochs):
            # à chaque époque, on met à jour W et B avec mini-batchs
            num_samples = y.shape[0]
            ret = num_samples % batch_size
            # si taille des données pas divisable par batch size alors resize les données
            if ret != 0:
                X_resize = X[:len(X)-ret]
                y_resize = y[:len(y)-ret]

            iterations = int(num_samples / batch_size)
            print(f"Nombre d'itérations par époque = {iterations}")
            for i in range(iterations):
                # pour chaque mini-batch on veut entrainer le modèle et mettre à jour les poids
                start = i * batch_size
                end = start+batch_size
                X_batch = X_resize[start:end, :]
                y_batch = y_resize[start:end, :]
                self.update(X_batch, y_batch, lr)

            # calcul de l'accuracy et de la loss pour cette époque 
            accuracy, loss = self.evaluate_model(X, y)
            print(f'Epoch {epoch}')
            print(f'Accuracy = {accuracy}')
            print(f'Loss = {loss}')
    
    def predict(self, X):
        """Prediction sur des nouvelles données"""
        preds = []
        for x in X:
            z, act = self.forward(x)
            preds.append(act)
        return np.array(preds)
    
    def evaluate_model(self, X, Y):
        """Calcule l'accuracy du modèle
        test_X = vecteur qui contient des X d'entrées
        test_Y = vecteur de vraie valeurs
        """
        preds = self.predict(X)
        loss = (1/(2*y.shape[0])) * np.linalg.norm(preds - y, 'fro') ** 2
        predicted = np.array([np.argmax(pred) for pred in preds])
        true_y = np.array([np.argmax(y) for y in Y])
        # si predicted == la vraie valeur (test_y) alors True sinon False 
        correct = predicted == true_y
        accuracy = (correct.sum() / float(Y.size))*100
        return accuracy, loss

    def write_output(self, X):
        preds = self.predict(X)
        predicted = np.array([np.argmax(pred) for pred in preds])
        with open('output.csv' ,'w', newline='') as fout:
            writer = csv.writer(fout)
            writer.writerow(["id", "class"])
            for i,pred in enumerate(predicted, start=1):
                writer.writerow([i, (pred+1)*100+1])

        


def sigmoid(z):
    """Fonction d'activation sigmoïde"""
    return 1 / (1 + np.exp(-z))
def sigmoid_prime(z):
    """Dérivée de la sigmoïde"""
    return sigmoid(z) * (1 - sigmoid(z))

In [58]:
net = Net(X.shape[1], y.shape[1])

In [ ]:
net.SGD_batch(X,y, 10, 100, 64)

In [59]:
net.write_output(X_to_predict)

C:\Users\Maxime\AppData\Local\Temp\ipykernel_13756\329248589.py:141: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-z))
